In [1]:
import pandas as pd

In [3]:
### Download from https://archive.ics.uci.edu/dataset/352/online+retail

path = "/content/drive/MyDrive/ICML-transactions/data_retail/Online Retail.xlsx"

df = pd.read_excel(path)

In [4]:
## Keep only United Kingdom transactions

df = df[df["Country"] == "United Kingdom"]

In [5]:
# take last 10% of data, split train, val and test

q_90 = df["InvoiceDate"].quantile(0.90)
q_98 = df["InvoiceDate"].quantile(0.98)
q_99 = df["InvoiceDate"].quantile(0.99)

df_train = df[(df["InvoiceDate"]  >= q_90) & (df["InvoiceDate"] < q_98)]
df_val = df[(df["InvoiceDate"]  >=q_98) & (df["InvoiceDate"] < q_99)]
df_test = df[df["InvoiceDate"] >= q_99]

In [22]:
for df in [df_train, df_val, df_test]:
  df['CustomerID'] = df['CustomerID'].astype('category')
  df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
  df["StockCode"] = df['StockCode'].astype('category')

# Create tasks

In [23]:
from enum import unique
def create_task_unique(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Adds a new column indicating whether values in `column` are unique.
    The new column is named `unique_<column>` and contains:
      - 1 if the value appears exactly once in the column
      - 0 otherwise
    """
    unique_col_name = f"unique_{column}"
    value_counts = df[column].value_counts()
    df[unique_col_name] = df[column].map(value_counts).eq(1).astype(int)
    return df

def create_task_count(df: pd.DataFrame, column: str, k: int, greater_than: bool = True) -> pd.DataFrame:
    """
    Adds a new column indicating whether values in `column` meet a count condition.

    The new column is named:
      - `count_gt_<k>_<column>` if greater_than is True
      - `count_eq_<k>_<column>` if greater_than is False

    Values:
      - 1 if the value appears more than k times (greater_than=True),
        otherwise exactly k times
      - 0 otherwise
    """
    suffix = "gt" if greater_than else "eq"
    new_col_name = f"count_{suffix}_{k}_{column}"

    value_counts = df[column].value_counts()

    if greater_than:
        mask = df[column].map(value_counts).gt(k)
    else:
        mask = df[column].map(value_counts).eq(k)

    df[new_col_name] = mask.astype(int)
    return df

import pandas as pd

def create_task_double(df: pd.DataFrame, col_1: str, col_2: str, anchor) -> pd.DataFrame:
    """
    Adds a new column indicating whether there exists ANOTHER row
    with the same value in `col_1` and with `col_2 == anchor`.

    The new column is named `double_<col_1>_<col_2>_<anchor>`.

    Values:
      - 1 if there exists another row with the same `col_1`
        and with `col_2 == anchor`
      - 0 otherwise
    """
    new_col_name = f"double_{col_1}_{col_2}_{anchor}"

    # Count how many anchor rows exist per col_1
    anchor_counts = (
        df[col_2].eq(anchor)
        .groupby(df[col_1])
        .sum()
    )

    # Map counts back to rows
    counts_per_row = df[col_1].map(anchor_counts).fillna(0)

    # Exists "other" row logic
    df[new_col_name] = (
        ((df[col_2] != anchor) & (counts_per_row >= 1)) |
        ((df[col_2] == anchor) & (counts_per_row >= 2))
    ).astype(int)

    return df


def create_task_diamond(df: pd.DataFrame, col_1: str, col_2: str) -> pd.DataFrame:
    """
    Adds a new column indicating whether there exists another row
    with the same (`col_1`, `col_2`) combination.

    The new column is named `duplicate_<col_1>_<col_2>`.

    Values:
      - 1 if the (`col_1`, `col_2`) pair appears more than once
      - 0 otherwise
    """
    new_col_name = f"duplicate_{col_1}_{col_2}"

    counts = df.groupby([col_1, col_2])[col_1].transform("size")
    df[new_col_name] = (counts > 1).astype(int)

    return df

In [24]:
for df in [df_train, df_val, df_test]:
  df = create_task_unique(df, 'StockCode')
  df = create_task_count(df, 'InvoiceNo', 75)
  df = create_task_count(df, 'InvoiceNo', 15, greater_than=False)
  df = create_task_double(df, 'InvoiceNo', 'StockCode', "23084")
  df = create_task_diamond(df, 'CustomerID', 'StockCode')

/tmp/ipython-input-296745467.py:88: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = df.groupby([col_1, col_2])[col_1].transform("size")
/tmp/ipython-input-296745467.py:88: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = df.groupby([col_1, col_2])[col_1].transform("size")
/tmp/ipython-input-296745467.py:88: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = df.groupby([col_1, col_2])[col_1].transform("si

In [ ]:
# Save df_train, df_val and df_test on drive

OUTPUT_PATH = 'data/processed'

df_train.to_csv(OUTPUT_PATH + 'retail-df_train.csv')
df_val.to_csv(OUTPUT_PATH + 'retail-df_val.csv')
df_test.to_csv(OUTPUT_PATH + 'retail-df_test.csv')